# 1. Parameters

In [1]:
from pathlib import Path

# Defaults
cases_dir = 'cases/full'
metadata_file = 'metadata.tsv'
iterations = 3
mincov = 10
ncores = 4

reference_files = {
    '0810PADBR-1': Path('data/reference/campylobacter/GCA_001879185.2_ASM187918v2_genomic.gbk.gz'),
    '1405WAEXK-1': Path('data/reference/ecoli/GCF_000703365.1_Ec2011C-3609_genomic.gbk.gz'),
    '1408MLGX6-3WGS': Path('data/reference/listeria/GCF_001047715.2_ASM104771v2_genomic.gbk.gz'),
    '1203NYJAP-1 - Tuna Scrape Outbreak': Path('data/reference/salmonella/GCF_000439415.1_ASM43941v1_genomic.gbk.gz'),
}
    
build_tree = True
sample_batch_size=10

In [2]:
from pathlib import Path
from shutil import rmtree
from os import makedirs
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)
index_path = cases_dir_path / 'index'

if cases_dir_path.exists():
    rmtree(cases_dir_path)
    
if not cases_dir_path.exists():
    makedirs(cases_dir_path)

case_name = str(cases_dir_path.name)

benchmark_path = cases_dir_path / 'index-info.tsv'

output_trees = {x: cases_dir_path / f'{x}_tree.tre'.replace(' ', '_') for x in reference_files}
output_trees

{'0810PADBR-1': PosixPath('cases/full/0810PADBR-1_tree.tre'),
 '1405WAEXK-1': PosixPath('cases/full/1405WAEXK-1_tree.tre'),
 '1408MLGX6-3WGS': PosixPath('cases/full/1408MLGX6-3WGS_tree.tre'),
 '1203NYJAP-1 - Tuna Scrape Outbreak': PosixPath('cases/full/1203NYJAP-1_-_Tuna_Scrape_Outbreak_tree.tre')}

# 2. Create subset inputs

In [3]:
import pandas as pd
from pathlib import Path
from typing import Dict

metadata_df = pd.read_csv(metadata_file, sep='\t')

def write_subset_input(metadata_df: pd.DataFrame, dataset_name: str) -> Path:
    all_input_total = len(metadata_df)
    cases_input = cases_dir_path / f'{dataset_name}_input-files-case.tsv'.replace(' ', '_')

    input_df = metadata_df.copy().loc[metadata_df['dataSetName'] == dataset_name]
    input_df['Sample'] = input_df['strain']
    input_df['Assemblies'] = pd.NA
    input_df['Reads1'] = input_df['Sample'].apply(lambda x: str((Path('data') / 'fastq' / (x + '_1.fastq.gz')).absolute()))
    input_df['Reads2'] = input_df['Sample'].apply(lambda x: str((Path('data') / 'fastq' / (x + '_2.fastq.gz')).absolute()))
    input_df = input_df[['Sample', 'Assemblies', 'Reads1', 'Reads2']]

    input_df.to_csv(cases_input, sep='\t', index=False)

    subset_input_total = len(input_df)

    print(f'Wrote dataset={dataset_name} consisting of {subset_input_total}/{all_input_total} samples to {cases_input}')
    
    return cases_input

cases_inputs = {x: write_subset_input(metadata_df, dataset_name=x) for x in reference_files}
cases_inputs

Wrote dataset=0810PADBR-1 consisting of 22/85 samples to cases/full/0810PADBR-1_input-files-case.tsv
Wrote dataset=1405WAEXK-1 consisting of 9/85 samples to cases/full/1405WAEXK-1_input-files-case.tsv
Wrote dataset=1408MLGX6-3WGS consisting of 31/85 samples to cases/full/1408MLGX6-3WGS_input-files-case.tsv
Wrote dataset=1203NYJAP-1 - Tuna Scrape Outbreak consisting of 23/85 samples to cases/full/1203NYJAP-1_-_Tuna_Scrape_Outbreak_input-files-case.tsv


{'0810PADBR-1': PosixPath('cases/full/0810PADBR-1_input-files-case.tsv'),
 '1405WAEXK-1': PosixPath('cases/full/1405WAEXK-1_input-files-case.tsv'),
 '1408MLGX6-3WGS': PosixPath('cases/full/1408MLGX6-3WGS_input-files-case.tsv'),
 '1203NYJAP-1 - Tuna Scrape Outbreak': PosixPath('cases/full/1203NYJAP-1_-_Tuna_Scrape_Outbreak_input-files-case.tsv')}

# 2. Index genomes

In [4]:
!gdi --version

gdi, version 0.6.0.dev2


## 2.1. Index reads

In [5]:
import time

start = time.time()
benchmarker = gdi_benchmark.IndexBenchmarkerMultiple(index_path=index_path, input_files_files=cases_inputs,
                                             reference_files=reference_files, mincov=mincov,
                                             build_tree=build_tree,
                                             ncores=ncores,
                                             sample_batch_size=sample_batch_size)
benchmark_df = benchmarker.benchmark(iterations=iterations)
end = time.time()
print(f'***Finished benchmarking, took {(end - start)/60:0.2f} minutes***')

Creating new index: [gdi init cases/full/index]


Creating a new index took 3.31 seconds



Iteration 1 of index/analysis of 22 samples for reference=data/reference/campylobacter/GCA_001879185.2_ASM187918v2_genomic.gbk.gz with 4 cores
Removing any extra snakemake directories: []
Analysis running: [gdi --project-dir cases/full/index --ncores 4 analysis --use-conda --no-load-data --reference-file data/reference/campylobacter/GCA_001879185.2_ASM187918v2_genomic.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/full/0810PADBR-1_input-files-case.tsv]


Analysis took 52.78 minutes
Index running: [gdi --project-dir cases/full/index --ncores 4 load vcf-kmer --sample-batch-size 10 --reference-file data/reference/campylobacter/GCA_001879185.2_ASM187918v2_genomic.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/reads/snakemake-assemblies.1645993922.9830651/gdi-input.fofn]


Indexing took 0.91 minutes
Building tree: [gdi --project-dir cases/full/index --ncores 4 rebuild tree --align-type full --extra-params '--fast -m GTR+F+R4' GCA_001879185.2_ASM187918v2_genomic]


...

Indexing took 5.31 minutes
Building tree: [gdi --project-dir cases/full/index --ncores 4 rebuild tree --align-type full --extra-params '--fast -m GTR+F+R4' GCF_000439415.1_ASM43941v1_genomic]


Building tree took 0.65 minutes


Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7f3eedb5ea60> but it is already set


A reminder to myself to look for a Python solution for directory sizes (instead of running `du`)


***Finished benchmarking, took 455.40 minutes***


In [6]:
benchmark_df

,Name,Reference name,Iteration,Number samples,Number features (all),Number features (no unknown),Number cores,Reference length,Analysis runtime,Analysis memory (max),...,Analysis disk uage,Index runtime,Index memory (max),Index memory (max/process),Index size,Tree runtime,Tree memory (max),Tree memory (max/process),Total runtime,Max memory
0,0810PADBR-1,GCA_001879185.2_ASM187918v2_genomic,1,22,45117,1102,4,1634890,3166.13,1.558306e+10,...,1.200301e+10,54.20,2.115244e+09,7.189709e+08,46587904.0,24.67,5.446205e+08,276631552.0,3245.00,1.558306e+10
0,1405WAEXK-1,GCF_000703365.1_Ec2011C-3609_genomic,1,9,550816,601,4,5412686,988.93,5.550060e+09,...,6.555070e+09,216.56,3.802116e+09,3.802116e+09,134742016.0,31.23,8.184218e+08,459071488.0,1236.72,5.550060e+09
0,1408MLGX6-3WGS,GCF_001047715.2_ASM104771v2_genomic,1,31,142442,170,4,2939733,2388.76,1.005870e+10,...,1.080715e+10,123.25,2.534011e+09,9.588531e+08,174067712.0,34.07,9.945620e+08,660180992.0,2546.08,1.005870e+10
0,1203NYJAP-1 - Tuna Scrape Outbreak,GCF_000439415.1_ASM43941v1_genomic,1,23,418622,165,4,4808805,1517.54,6.843236e+09,...,9.016287e+09,300.64,5.624377e+09,2.788295e+09,251715584.0,38.29,1.335677e+09,870260736.0,1856.47,6.843236e+09
0,0810PADBR-1,GCA_001879185.2_ASM187918v2_genomic,2,22,45117,1102,4,1634890,3158.43,1.560137e+10,...,1.200307e+10,56.88,2.290647e+09,7.244390e+08,46682112.0,21.59,5.444198e+08,276242432.0,3236.90,1.560137e+10
0,1405WAEXK-1,GCF_000703365.1_Ec2011C-3609_genomic,2,9,550816,601,4,5412686,991.97,5.480444e+09,...,6.555058e+09,217.37,3.771884e+09,3.771884e+09,134828032.0,30.81,8.194007e+08,459337728.0,1240.15,5.480444e+09
0,1408MLGX6-3WGS,GCF_001047715.2_ASM104771v2_genomic,2,31,142442,170,4,2939733,2388.61,9.555489e+09,...,1.080714e+10,117.73,3.185816e+09,1.059996e+09,173850624.0,36.21,9.951846e+08,660570112.0,2542.55,9.555489e+09
0,1203NYJAP-1 - Tuna Scrape Outbreak,GCF_000439415.1_ASM43941v1_genomic,2,23,418622,165,4,4808805,2102.23,6.751625e+09,...,9.016263e+09,307.78,6.020960e+09,2.700640e+09,252108800.0,37.54,1.335370e+09,870191104.0,2447.55,6.751625e+09
0,0810PADBR-1,GCA_001879185.2_ASM187918v2_genomic,3,22,45117,1102,4,1634890,3164.05,1.538114e+10,...,1.200303e+10,57.25,2.191090e+09,7.395000e+08,46538752.0,22.05,5.440676e+08,276090880.0,3243.35,1.538114e+10
0,1405WAEXK-1,GCF_000703365.1_Ec2011C-3609_genomic,3,9,550816,601,4,5412686,986.85,5.426352e+09,...,6.555066e+09,229.27,3.808711e+09,3.808711e+09,134664192.0,29.04,8.182333e+08,459100160.0,1245.16,5.426352e+09


In [7]:
benchmark_df.to_csv(benchmark_path, sep='\t', index=False)

# 3. Export trees

In [8]:
if build_tree:
    # Using variables with shell commands in Jupyter isn't working for me so I have to avoid using them
    !gdi --project-dir cases/full/index export tree GCA_001879185.2_ASM187918v2_genomic > cases/full/0810PADBR-1_tree.tre
    !gdi --project-dir cases/full/index export tree GCF_000703365.1_Ec2011C-3609_genomic > cases/full/1405WAEXK-1_tree.tre
    !gdi --project-dir cases/full/index export tree GCF_001047715.2_ASM104771v2_genomic > cases/full/1408MLGX6-3WGS_tree.tre
    !gdi --project-dir cases/full/index export tree GCF_000439415.1_ASM43941v1_genomic > cases/full/1203NYJAP-1_-_Tuna_Scrape_Outbreak_tree.tre
else:
    print(f'build_tree={build_tree} so no trees to export')